In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, RobustScaler, FunctionTransformer
import category_encoders as ce

In [38]:
import dagshub
dagshub.init(repo_owner='sourav664', repo_name='real-estate-hybrid-app', mlflow=True)


Accessing as sourav664

Initialized MLflow to track repo "sourav664/real-estate-hybrid-app"

Repository sourav664/real-estate-hybrid-app initialized!

In [39]:
import mlflow
mlflow.set_tracking_uri("https://dagshub.com/sourav664/real-estate-hybrid-app.mlflow")

In [4]:
print(mlflow.__version__)

2.22.0


In [5]:
# mlflow experiment

mlflow.set_experiment("Exp 3 - LightGBM Hyperparameter Tuning")

<Experiment: artifact_location='mlflow-artifacts:/831f39d69e9440f590c44c0ad5d152b2', creation_time=1764742211921, experiment_id='2', last_update_time=1764742211921, lifecycle_stage='active', name='Exp 3 - LightGBM Hyperparameter Tuning', tags={}>

In [25]:

from sklearn import set_config

set_config(transform_output="pandas")

In [1]:
from pathlib import Path

path = Path.cwd().parent
data_path = path / 'data/raw'

In [3]:
import pandas as pd
df_one = pd.read_csv(f"{data_path}/real_estate.csv")

In [4]:
predictors = ['propertytype', 'bedrooms', 'bathrooms', 'balconies', 'furnished', 'transactiontype', 'ageofcons', 'additionalRooms','region',
              'locality','superbuiltupareasqft','totalfloornumber','price']

In [5]:
df = df_one[predictors]

In [6]:
df['superbuiltupareasqft'].max()

np.float64(18900.0)

In [8]:
df_one.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41860 entries, 0 to 41859
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   propertyid            41860 non-null  int64  
 1   propertytype          41860 non-null  object 
 2   bedrooms              41860 non-null  float64
 3   bathrooms             41860 non-null  float64
 4   balconies             41860 non-null  float64
 5   furnished             41860 non-null  object 
 6   transactiontype       41860 non-null  object 
 7   ageofcons             41860 non-null  object 
 8   additionalRooms       41860 non-null  int64  
 9   region                41860 non-null  object 
 10  locality              41860 non-null  object 
 11  superbuiltupareasqft  41860 non-null  float64
 12  price                 41860 non-null  float64
 13  latitude              41860 non-null  float64
 14  longitude             41860 non-null  float64
 15  url                

In [6]:
df.duplicated().sum(
    
)

np.int64(2130)

In [7]:
df.drop_duplicates(inplace=True)

C:\Users\soura\AppData\Local\Temp\ipykernel_12192\3006716147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [12]:
df_one['price_per_sqft'] = round((df['price']/df['superbuiltupareasqft'])* 10000000)

In [14]:
df_one.to_csv(f"{data_path}/real_estate.csv", index=False)

In [8]:
df['propertytype'].value_counts()

propertytype
multistorey apartment      26206
builder floor apartment    10762
residential house           1626
villa                        992
penthouse                    144
Name: count, dtype: int64

In [9]:
X = df.drop(columns=["price"])
y = df["price"]

In [10]:
# train test split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
     

In [11]:
df.shape

(39730, 13)

In [12]:
print("The size of train data is",X_train.shape)
print("The shape of test data is",X_test.shape)

The size of train data is (31784, 12)
The shape of test data is (7946, 12)


In [13]:
ohe_encode = ['transactiontype','region','propertytype','furnished','ageofcons']
target_encode = ['locality']
robust_scaler = ['bedrooms','bathrooms','balconies','superbuiltupareasqft', 'totalfloornumber']

In [14]:
from sklearn.preprocessing import FunctionTransformer

pt = FunctionTransformer(np.log1p, inverse_func=np.expm1, validate=True)


In [15]:
preprocessor = ColumnTransformer(
    [
        ('ohe_encode', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), ohe_encode),
        ('robust_scaler', RobustScaler(), robust_scaler),
    ],remainder="passthrough", n_jobs=-1, force_int_remainder_cols=False,verbose_feature_names_out=False
)


preprocessor.set_output(transform="pandas")

,transformers,"[('ohe_encode', ...), ('robust_scaler', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,-1
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False
,force_int_remainder_cols,False
,categories,'auto'
,drop,'first'
,sparse_output,False


In [16]:
from lightgbm import LGBMRegressor
import optuna
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.compose import TransformedTargetRegressor

c:\Users\soura\OneDrive\Desktop\real-estate-hybrid-app\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [160]:
def objective(trial):
    with mlflow.start_run(nested=True):
        params = {
            "n_estimators": trial.suggest_int("n_estimators",10,500),
            "max_depth": trial.suggest_int("max_depth",1,40),
            "learning_rate": trial.suggest_float("learning_rate",0.01,0.8),
            "subsample": trial.suggest_float("subsample",0.5,1),
            "min_child_weight": trial.suggest_int("min_child_weight",5,20),
            "min_split_gain": trial.suggest_float("min_split_gain",0,10),
            "reg_lambda": trial.suggest_float("reg_lambda",0.1,100),
            "random_state": 42,
            "n_jobs": -1,
        }

        # log model parameters
        mlflow.log_params(params)

        lgb_reg = LGBMRegressor(**params)
        model = TransformedTargetRegressor(regressor=lgb_reg,transformer=pt)

        # # train the model
        # model.fit(X_train_trans,y_train)

        # # get the predictions
        # y_pred_train = model.predict(X_train_trans)
        # y_pred_test = model.predict(X_test_trans)
        
        
        
         

        full_pipe = Pipeline([
            ('target_encoder', ce.TargetEncoder(cols=target_encode)),
            ('preprocessor', preprocessor),
            ('model', model),
        ])



        # perform cross validation
        cv_score = cross_val_score(full_pipe,
                                X_train,
                                y_train,
                                cv=5,
                                scoring="neg_mean_absolute_error",
                                n_jobs=-1)

        # mean score
        mean_score = -(cv_score.mean())
        # log avg cross val error
        mlflow.log_metric("cross_val_error",mean_score)

        return mean_score

In [161]:
# create optuna study
study = optuna.create_study(direction="minimize")

with mlflow.start_run(run_name="best_model"):
    # optimize the objective function
    study.optimize(objective,n_trials=100,n_jobs=-1,show_progress_bar=True)

    # log the best parameters
    mlflow.log_params(study.best_params)

    # log the best score
    mlflow.log_metric("best_score",study.best_value)

    # train the model on best parameters
    best_lgbm = LGBMRegressor(**study.best_params)
    model = TransformedTargetRegressor(regressor=best_lgbm,transformer=pt)

    full_pipe = Pipeline([
            ('target_encoder', ce.TargetEncoder(cols=target_encode)),
            ('preprocessor', preprocessor),
            ('model', model),
        ])
    

    # train the model
    full_pipe.fit(X_train,y_train)

    # get the predictions
    y_pred_train = full_pipe.predict(X_train)
    y_pred_test = full_pipe.predict(X_test)
    
    
    

    scores = cross_val_score(full_pipe,
                         X_train,
                         y_train,
                         scoring="neg_mean_absolute_error",
                         cv=5,n_jobs=-1)

    # log metrics
    mlflow.log_metric("training_error",mean_absolute_error(y_train,y_pred_train))
    mlflow.log_metric("test_error",mean_absolute_error(y_test,y_pred_test))
    mlflow.log_metric("training_r2",r2_score(y_train,y_pred_train))
    mlflow.log_metric("test_r2",r2_score(y_test,y_pred_test))
    mlflow.log_metric("cross_val",- scores.mean())

    # log the best model
    mlflow.sklearn.log_model(best_lgbm,artifact_path="model")

[I 2025-12-12 12:56:56,171] A new study created in memory with name: no-name-d82fd1f7-25c0-433d-a8e5-422ac7bbc78d
  0%|          | 0/100 [00:00<?, ?it/s]

🏃 View run fearless-goose-745 at: https://dagshub.com/sourav664/real-estate-hybrid-app.mlflow/#/experiments/0/runs/1e933dd63dca4d37a7f0d4aac981dac2
🧪 View experiment at: https://dagshub.com/sourav664/real-estate-hybrid-app.mlflow/#/experiments/0


Best trial: 1. Best value: 0.892869:   1%|          | 1/100 [00:13<22:25, 13.59s/it]

[I 2025-12-12 12:57:11,326] Trial 1 finished with value: 0.8928691751444664 and parameters: {'n_estimators': 310, 'max_depth': 28, 'learning_rate': 0.3280994191838662, 'subsample': 0.510822194086112, 'min_child_weight': 5, 'min_split_gain': 3.0310065693164256, 'reg_lambda': 95.20670808625502}. Best is trial 1 with value: 0.8928691751444664.
🏃 View run salty-ray-335 at: https://dagshub.com/sourav664/real-estate-hybrid-app.mlflow/#/experiments/0/runs/2a07e1a27bec4ff395542ec2a65170f2
🧪 View experiment at: https://dagshub.com/sourav664/real-estate-hybrid-app.mlflow/#/experiments/0


Best trial: 1. Best value: 0.892869:   2%|▏         | 2/100 [00:15<10:51,  6.65s/it]

...

[I 2025-12-12 13:10:22,199] Trial 87 finished with value: 0.6708065789150195 and parameters: {'n_estimators': 476, 'max_depth': 28, 'learning_rate': 0.558318693338763, 'subsample': 0.9524910144553778, 'min_child_weight': 10, 'min_split_gain': 0.0067868341793319124, 'reg_lambda': 67.9433325154547}. Best is trial 70 with value: 0.6406000062335963.


c:\Users\soura\OneDrive\Desktop\real-estate-hybrid-app\.venv\Lib\site-packages\sklearn\compose\_column_transformer.py:975: FutureWarning: The parameter `force_int_remainder_cols` is deprecated and will be removed in 1.9. It has no effect. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\soura\OneDrive\Desktop\real-estate-hybrid-app\.venv\Lib\site-packages\sklearn\preprocessing\_function_transformer.py:314: UserWarning: When `set_output` is configured to be 'pandas', `func` should return a pandas DataFrame to follow the `set_output` API  or `feature_names_out` should be defined.
  warnings.warn(warn_msg.format("pandas"))
c:\Users\soura\OneDrive\Desktop\real-estate-hybrid-app\.venv\Lib\site-packages\sklearn\preprocessing\_function_transformer.py:314: UserWarning: When `set_output` is configured to be 'pandas', `func` should return a pandas DataFrame to follow the `set_output` API  or `feature_names_out` should be defined.
  warnings.warn(warn_msg.format("pan

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003087 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 594
[LightGBM] [Info] Number of data points in the train set: 31784, number of used features: 23
[LightGBM] [Info] Start training from score 1.337475
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet t

2025/12/12 13:10:40 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/12/12 13:10:40 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run best_model at: https://dagshub.com/sourav664/real-estate-hybrid-app.mlflow/#/experiments/0/runs/cf9e3123eb834f77878b3dd77b7238a7
🧪 View experiment at: https://dagshub.com/sourav664/real-estate-hybrid-app.mlflow/#/experiments/0


In [110]:
study.best_params

{'n_estimators': 367,
 'max_depth': 23,
 'learning_rate': 0.41240289974488187,
 'subsample': 0.5959287201715322,
 'min_child_weight': 16,
 'min_split_gain': 0.005146552081925616,
 'reg_lambda': 21.39843889879577}

In [28]:
lgbm_params = {'n_estimators': 498,
 'max_depth': 9,
 'learning_rate': 0.07041175194094815,
 'subsample': 0.9404689293367969,
 'min_child_weight': 6,
 'min_split_gain': 0.0004206922397676019,
 'reg_lambda': 14.050384510957326}

In [184]:
study.best_params

{'n_estimators': 497,
 'max_depth': 18,
 'learning_rate': 0.3411567207702695,
 'subsample': 0.8211191192401062,
 'min_child_weight': 9,
 'min_split_gain': 0.001980136353254995,
 'reg_lambda': 66.24757875556628}

In [17]:
lgbm_params1 = {'n_estimators': 497,
 'max_depth': 18,
 'learning_rate': 0.3411567207702695,
 'subsample': 0.8211191192401062,
 'min_child_weight': 9,
 'min_split_gain': 0.001980136353254995,
 'reg_lambda': 66.24757875556628}

In [34]:
lgbm_params2 = {'n_estimators': 367,
 'max_depth': 23,
 'learning_rate': 0.41240289974488187,
 'subsample': 0.5959287201715322,
 'min_child_weight': 16,
 'min_split_gain': 0.005146552081925616,
 'reg_lambda': 21.39843889879577}

In [35]:
best_lgbm = LGBMRegressor(**lgbm_params2)
model = TransformedTargetRegressor(regressor=best_lgbm,transformer=pt)


full_pipe = Pipeline([
            ('target_encoder', ce.TargetEncoder(cols=target_encode)),
            ('preprocessor', preprocessor),
            ('model', model),
        ])

In [36]:
full_pipe.fit(X_train,y_train)

c:\Users\soura\OneDrive\Desktop\real-estate-hybrid-app\.venv\Lib\site-packages\sklearn\compose\_column_transformer.py:975: FutureWarning: The parameter `force_int_remainder_cols` is deprecated and will be removed in 1.9. It has no effect. Leave it to its default value to avoid this warning.
  warnings.warn(


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001372 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 594
[LightGBM] [Info] Number of data points in the train set: 31784, number of used features: 23
[LightGBM] [Info] Start training from score 1.337475
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped trainin

,steps,"[('target_encoder', ...), ('preprocessor', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,verbose,0
,cols,['locality']
,drop_invariant,False
,return_df,True
,handle_missing,'value'
,handle_unknown,'value'
,min_samples_leaf,20


In [37]:
y_pred_train = full_pipe.predict(X_train)
y_pred_test = full_pipe.predict(X_test)

In [38]:
print(f"The train error is {mean_absolute_error(y_train,y_pred_train):.2f} cr")
print(f"The test error is {mean_absolute_error(y_test,y_pred_test):.2f} cr")

The train error is 0.52 cr
The test error is 0.60 cr


In [39]:
print(f"The train r2 score is {r2_score(y_train,y_pred_train):.2f}")
print(f"The test r2 score is {r2_score(y_test,y_pred_test):.2f}")

The train r2 score is 0.93
The test r2 score is 0.88


In [23]:
# Creating a pipeline
pipeline = Pipeline([
    ('target_encoder', ce.TargetEncoder(cols=target_encode)),
    ('preprocessor', preprocessor)
    
])

In [24]:
pipeline.fit(X_train,y_train)

c:\Users\soura\OneDrive\Desktop\real-estate-hybrid-app\.venv\Lib\site-packages\sklearn\compose\_column_transformer.py:975: FutureWarning: The parameter `force_int_remainder_cols` is deprecated and will be removed in 1.9. It has no effect. Leave it to its default value to avoid this warning.
  warnings.warn(


,steps,"[('target_encoder', ...), ('preprocessor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,verbose,0
,cols,['locality']
,drop_invariant,False
,return_df,True
,handle_missing,'value'
,handle_unknown,'value'
,min_samples_leaf,20


In [25]:
X_train_trans = pipeline.transform(X_train)
X_test_trans = pipeline.transform(X_test)

In [26]:
best_lgbm = LGBMRegressor(**study.best_params)
model = TransformedTargetRegressor(regressor=best_lgbm,transformer=pt)

model.fit(X_train_trans,y_train)

NameError: name 'study' is not defined

In [195]:
y_pred_train = model.predict(X_train_trans)
y_pred_test = model.predict(X_test_trans)

In [196]:
print(f"The train error is {mean_absolute_error(y_train,y_pred_train):.2f} cr")
print(f"The test error is {mean_absolute_error(y_test,y_pred_test):.2f} cr")

The train error is 0.50 cr
The test error is 0.60 cr


In [197]:
print(f"The train r2 score is {r2_score(y_train,y_pred_train):.2f}")
print(f"The test r2 score is {r2_score(y_test,y_pred_test):.2f}")

The train r2 score is 0.93
The test r2 score is 0.89
